In [1]:
# Create Variables
$Password = "Passw0rd!"
$UserName = "testuser" # Can be also domain\testuser
$BasePath = "C:\GIT\Presentations\PowerShellUserGroup-2020-11-05"
$KeyPath = Join-Path -Path $BasePath -ChildPath "AES.key"
$PasswordPath = Join-Path -Path $BasePath -ChildPath "Password.txt"
$CLIPathIntegrated = Join-Path -Path $BasePath -ChildPath "CLIIntegrated.xml"
$CLIPathPSFramework = Join-Path -Path $BasePath -ChildPath "CLIPSFramework.xml"

$Password
$UserName
$BasePath
$KeyPath
$PasswordPath
$CLIPathIntegrated 
$CLIPathPSFramework

# PowerShell User Group innsalzach 05.11.2020

## Handling and managing Secrets using PowerShell

## Basics of PowerShell Credentials

If you want to create a secret you need to things:  

- a UserName which is plain text (you can replace UserName with anything else)
- the password (needs to be a secure string)(the password could also be an API KEY or something else you want to store as secure string)

## Passwords in an interactive Shell

You can also enter the Password like this:

Problem: What do you do if this is used in a pipeline or scheduled task for example?

Only applicable for interactive scripts.

In [ ]:
# Interactive Password
$InteractivePassword = Read-Host -AsSecureString -Prompt "Please Enter Password"
$InteractivePassword

## If you want to use the PSCredential Object only on this computer use the following example:

### Create a secure string

In [ ]:
# Create a secure string
$SecPassword = ConvertTo-SecureString -String $Password -AsPlainText -Force

Let's look at the return value

In [ ]:
# Returnvalue
$SecPassword

Get The password again in clear Text

In [ ]:

ConvertFrom-SecureString -SecureString $SecPassword -AsPlainText

### Create a PSCredential Object using a securestring obejct

**Note**: The SecPassword Variable has to be always of type System.Security.SecureString. Please do not put a string an Argument to the PSCredential Object.

**Reference in the Microsoft Docs**: [PSCredential Object](https://docs.microsoft.com/en-us/dotnet/api/system.management.automation.pscredential?view=powershellsdk-7.0.0)

In [ ]:

$splat = @{
    TypeName = "System.Management.Automation.PSCredential"
    ArgumentList = $UserName, $SecPassword
}

$PSCred = New-Object @splat 
$PSCred

Create a PSCredential Object in an interactive way

In [ ]:

$Cred = Get-Credential -Message "Please enter your password"

If you want to reuse that credential you can export it to a file in an encrypted way

In [ ]:

# PowerShell integrated way
$Cred | Export-CliXML -Path $CLIPathIntegrated

# PSFramewor way
$Cred | Export-PSFClixml -Path $CLIPathPSFramework

Import the XML Files whenever neeeded

**Note:** This can only be done on the same machine

In [ ]:

# Import Credential using the integrated way
Import-Clixml -Path $CLIPathIntegrated

# Import Credential using PSFramework
Import-PSFClixml -Path $CLIPathPSFramework

## If you want to share a PSCredential with remote Computers also please use a key.  

**Note**: You can use that also if you want to use It only on one computer.

## Create a Key File

**Note**: If you want to make that key accessible to more computers store It somewhere safe on a fileshare. (Take care of the right ACLs)

In [ ]:

$KeyPath = Join-Path -Path $BasePath -ChildPath "AES.key"
$Key = New-Object -TypeName byte[] 16
[Security.Cryptography.RNGCryptoServiceProvider]::Create().GetBytes($Key)
$key | Out-File -FilePath $KeyPath

###  Create a SecureString and encrypt It with the key

In [ ]:

$key = Get-Content -Path $KeyPath
$EncPassword = ConvertTo-SecureString -String $Password -AsPlainText -Force
$EncPassword | ConvertFrom-SecureString -Key $key | Out-File -FilePath $PasswordPath

Reminder: Open Password File

### Create the PSCredential Object

In [ ]:

$key = Get-Content -Path $KeyPath
$pwd = Get-Content -Path $PasswordPath | ConvertTo-SecureString -Key $key
$PSCred = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList $UserName, $pwd

## Secrets Management and KeePass

KeePass ([KeePass Password Safe](https://keepass.info/)) is a widely uses Password Management System. It stores all Passwords into a .kdbx File.

In a .kdbx File you can have folders and Passwords. You can organize the passwords in this folders.

If you want to open the .kdbx File you have to provide a master password.

To access KeePass via PowerShell you have to install a module called [GitHub - PSKeePass/PoShKeePass: PowerShell module for KeePass](https://github.com/PSKeePass/PoShKeePass)

This module is only available in PowerShell 5.1 (Use PowerShell 5.1 for that)

In [1]:
Install-Module -Name PoShKeePass

The below script needs to be able to find the current output cell; this is an easy method to get it.

Let's take a look what Cmdlets do come with the module

In [ ]:

Get-Command -Module PoShKeePass

 So Let's create our KeePass Database and use a KeyFile for that

In [ ]:

New-KeePassDatabase -DatabasePath C:\PSUG\KeePass\PSCreateKeepass.kdbx -KeyPath C:\PSUG\KeePass\PSUGTest.key

Snap! That's not implemented yet. If you want to create that you have to do that through the GUI (Show that).

If you want to create a KeePass Database which is only protected by a master key you can create that with PowerShell

In [ ]:

$Credential = Get-Credential -UserName "NotNeeded" -Message "Please Provide the KeePass Master Key"
New-KeePassDatabase -DatabasePath C:\PSUG\KeePass\PSUGDatabaseWithMasterKeyOnly.kdbx -MasterKey $Credential

Now you can open the Database with PowerShell. To open the KeePass Database you first have to create a KeePass Database Configuration.

So Let's do that

In [ ]:

Get-KeePassDatabaseConfiguration | Remove-KeePassDatabaseConfiguration
New-KeePassDatabaseConfiguration -DatabaseProfileName PSUGTest -DatabasePath C:\PSUG\KeePass\Database.kdbx -KeyPath C:\PSUG\KeePass\PSUGTest.key
Get-KeePassDatabaseConfiguration

If you want to create a KeePass Database Configuration with a Master Key protected KeePass file

In [ ]:

Get-KeePassDatabaseConfiguration | Remove-KeePassDatabaseConfiguration
New-KeePassDatabaseConfiguration -DatabaseProfileName PSUGTest -DatabasePath C:\PSUG\KeePass\Database.kdbx -UseMasterKey
Get-KeePassDatabaseConfiguration

  

You can have multiple Database Configurations but if you want to set one as default

In [ ]:

Get-KeePassDatabaseConfiguration | Remove-KeePassDatabaseConfiguration
New-KeePassDatabaseConfiguration -DatabaseProfileName PSUGTest -DatabasePath C:\PSUG\KeePass\Database.kdbx -UseMasterKey -Default
Get-KeePassDatabaseConfiguration

If you want to set the default switch on a existing Database Configuration use (There is tab completion on the DatabaseProfileName)

In [ ]:

Update-KeePassDatabaseConfiguration -DatabaseProfileName PSUGTest -Default

Now we have to get the Root of our KeePass Database (With Master Key)

In [ ]:

$MasterKey = ConvertTo-SecureString -String ($Credential.GetNetworkCredential().Password) -AsPlainText -Force
$rootPath = (Get-KeePassGroup -MasterKey $MasterKey | Where-Object {-not $_.ParentGroup}).Name

Now we have to get the Root of our KeePass Database (With Key File)

In [ ]:

$rootPath = (Get-KeePassGroup -MasterKey $Credential.GetNetworkCredential().Password | Where-Object {-not $_.ParentGroup}).Name

To Create a new Folder inside our KeePass Database (With Master Key)

In [ ]:

$MasterKey = ConvertTo-SecureString -String ($Credential.GetNetworkCredential().Password) -AsPlainText -Force
New-KeePassGroup -KeePassGroupParentPath $rootPath -KeePassGroupName "PSUG Folder" -MasterKey $MasterKey -ErrorAction SilentlyContinue

To Create a new Folder inside our KeePass Database (With Key File)

In [ ]:

New-KeePassGroup -KeePassGroupParentPath $rootPath -KeePassGroupName "PSUG Folder"

To Verify that we have created the folder inside our KeePass Database (With Master Key)

In [ ]:

$MasterKey = ConvertTo-SecureString -String ($Credential.GetNetworkCredential().Password) -AsPlainText -Force
Get-KeePassGroup -KeePassGroupPath "$rootPath/PSUG Folder" -MasterKey $MasterKey -ErrorAction SilentlyContinue

To Verify that we have created the folder inside our KeePass Database (With Key File)

In [ ]:

Get-KeePassGroup -KeePassGroupPath "$rootPath/PSUG Folder"

To Create a KeePass Password (With Master Key)

In [ ]:

$Passsword = Read-Host -AsSecureString -Prompt "Please enter the password"
New-KeePassEntry -KeePassEntryGroupPath $rootPath -Title "TestEntry" -UserName "TestUser" -KeePassPassword $Passsword -MasterKey $MasterKey -ErrorAction SilentlyContinue

To Create a KeePass Password (With Key File)

In [ ]:

$Passsword = Read-Host -AsSecureString -Prompt "Please enter the password"
New-KeePassEntry -KeePassEntryGroupPath $rootPath -Title "TestEntry" -UserName "TestUser" -KeePassPassword $Passsword

To Check if we realy created that entry (With Master Key)

In [ ]:

Get-KeePassEntry -KeePassEntryGroupPath $rootPath -Title TestEntry -MasterKey $MasterKey -ErrorAction SilentlyContinue

To Check if we realy created that entry (With Key File)

In [ ]:

Get-KeePassEntry -KeePassEntryGroupPath $rootPath -Title TestEntry

To get the entry back as PSCredential (With Master Key)

In [ ]:

$entry = Get-KeePassEntry -KeePassEntryGroupPath $rootPath -Title TestEntry -MasterKey $MasterKey -ErrorAction SilentlyContinue
$entry | Get-Member

$EntryToPSCred = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList $entry.UserName, $entry.Password
$EntryToPSCred | Get-Member

To get the entry back as PSCredential (With Key File)

In [ ]:

$entry = Get-KeePassEntry -KeePassEntryGroupPath $rootPath -Title TestEntry
$entry | Get-Member

$EntryToPSCred = New-Object -TypeName System.Management.Automation.PSCredential -ArgumentList $entry.UserName, $entry.Password
$EntryToPSCred | Get-Member

TODO: Azure KeyVault, LastPass, Secret Management

## Housekeeping

In [2]:

Remove-Item -Path $KeyPath -Force
Remove-Item -Path $PasswordPath -Force
Remove-Item -Path $CLIPathIntegrated -Force
Remove-Item -Path $CLIPathPSFramework -Force

Get-KeePassDatabaseConfiguration | Remove-KeePassDatabaseConfiguration
Remove-Item -Path C:\PSUG -Recurse -Force